### Import necessary libraries and modules

In [ ]:
from IPython.display import clear_output
import pandas as pd
import os, shutil
from sklearn.model_selection import train_test_split

!zenodo_get https://doi.org/10.5281/zenodo.10430727
!unzip PCOSGen-train.zip
!mv PCOSGen-train/images PCOSGen-train/Training
!mkdir data data/PCOS
!mv PCOSGen-train/Training data/PCOS
!mv PCOSGen-train/class_label.xlsx class_label.xlsx

labels_df = pd.read_excel('class_label.xlsx')
os.makedirs('data/PCOS/Training/0', exist_ok=True)
os.makedirs('data/PCOS/Training/1', exist_ok=True)
os.makedirs('data/PCOS/Testing', exist_ok=True)
os.makedirs('data/PCOS/Testing/0', exist_ok=True)
os.makedirs('data/PCOS/Testing/1', exist_ok=True)

for index, row in labels_df.iterrows():
    image_path = os.path.join('data/PCOS/Training', row['imagePath'])
    destination_dir = os.path.join('data/PCOS/Training', str(row['Healthy']))
    destination_path = os.path.join(destination_dir, row['imagePath'])
    shutil.move(image_path, destination_path)

for label in [0, 1]:
    images = os.listdir(f'data/PCOS/Training/{label}')
    train_images, test_images = train_test_split(images, test_size=0.1, random_state=42)

    for image in test_images:
        src_path = f'data/PCOS/Training/{label}/{image}'
        dst_path = f'data/PCOS/Testing/{label}/{image}'
        shutil.move(src_path, dst_path)    
    
!rm -rf PCOSGen-train Meta_Data.pdf PCOSGen-train.zip md5sums.txt class_label.xlsx

clear_output()
print("Images have been successfully moved to the respective label directories.")
train_count_0 = len(os.listdir('data/PCOS/Training/0'))
train_count_1 = len(os.listdir('data/PCOS/Training/1'))
test_count_0 = len(os.listdir('data/PCOS/Testing/0'))
test_count_1 = len(os.listdir('data/PCOS/Testing/1'))

print(f"Training set count for class 0: {train_count_0}")
print(f"Training set count for class 1: {train_count_1}")
print(f"Testing set count for class 0: {test_count_0}")
print(f"Testing set count for class 1: {test_count_1}")

Images have been successfully moved to the respective label directories.
Training set count for class 0: 2067
Training set count for class 1: 812
Testing set count for class 0: 230
Testing set count for class 1: 91


In [2]:
import os
import pickle
import time
from collections import OrderedDict
from typing import (
    List, Tuple, Dict, Optional, Callable, Union
)
import tenseal as ts

import numpy as np
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import flwr as fl
from flwr.common import (
    Metrics, EvaluateIns, EvaluateRes, FitIns, FitRes, MetricsAggregationFn, 
    Scalar, logger, ndarrays_to_parameters_custom, parameters_to_ndarrays_custom,
    Parameters, NDArrays
)
from flwr.server.client_proxy import ClientProxy
from flwr.server.client_manager import ClientManager
from flwr.server.strategy.aggregate import weighted_loss_avg
from logging import WARNING
import pennylane as qml

from utils import *

### Creation of FHE Keys

In [3]:
def combo_keys(client_path="secret.pkl", server_path="server_key.pkl"):
    """
    To create the public/private keys combination
    args:
        client_path: path to save the secret key (str)
        server_path: path to save the server public key (str)
    """
    context_client = security.context()
    security.write_query(client_path, {"contexte": context_client.serialize(save_secret_key=True)})
    security.write_query(server_path, {"contexte": context_client.serialize()})

    _, context_client = security.read_query(client_path)
    _, context_server = security.read_query(server_path)

    context_client = ts.context_from(context_client)
    context_server = ts.context_from(context_server)
    print("Is the client context private?", ("Yes" if context_client.is_private() else "No"))
    print("Is the server context private?", ("Yes" if context_server.is_private() else "No"))


secret_path = "secret.pkl"
public_path = "server_key.pkl"
if os.path.exists(secret_path):
    print("it exists")
    _, context_client = security.read_query(secret_path)

else:
    combo_keys(client_path=secret_path, server_path=public_path)

n_qubits = 4
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}       

it exists


### Model Architecture Creation

In [4]:
dev = qml.device("default.qubit.torch", wires=n_qubits)
    
@qml.qnode(dev, interface='torch')
def quantum_net(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits)) 
    qml.BasicEntanglerLayers(weights,wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

class Net(nn.Module):
    """
    A simple CNN model

    Args:
        num_classes: An integer indicating the number of classes in the dataset.
    """
    def __init__(self, num_classes=10) -> None:
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 * 56 * 56, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, n_qubits),
            qml.qnn.TorchLayer(quantum_net, weight_shapes=weight_shapes),
            nn.Linear(n_qubits, num_classes)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of the neural network
        """
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

### Define the FlowerClient class for federated learning

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader, device, batch_size, save_results, matrix_path, roc_path,
                 yaml_path, he, classes, context_client):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.cid = cid
        self.device = device
        self.batch_size = batch_size
        self.save_results = save_results
        self.matrix_path = matrix_path
        self.roc_path = roc_path
        self.yaml_path = yaml_path
        self.he = he
        self.classes = classes
        self.context_client = context_client

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters2(self.net, self.context_client)

    def fit(self, parameters, config):
        server_round = config['server_round']
        local_epochs = config['local_epochs']
        lr = float(config["learning_rate"])

        print(f'[Client {self.cid}, round {server_round}] fit, config: {config}')

        set_parameters(self.net, parameters, self.context_client)

        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.net.parameters(), lr=lr)

        results = engine.train(self.net, self.trainloader, self.valloader, optimizer=optimizer, loss_fn=criterion,
                               epochs=local_epochs, device=self.device)

        if self.save_results:
            save_graphs(self.save_results, local_epochs, results, f"_Client {self.cid}")

        return get_parameters2(self.net, self.context_client), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters, self.context_client)

        loss, accuracy, y_pred, y_true, y_proba = engine.test(self.net, self.valloader,
                                                              loss_fn=torch.nn.CrossEntropyLoss(), device=self.device)

        if self.save_results:
            os.makedirs(self.save_results, exist_ok=True)
            if self.matrix_path:
                save_matrix(y_true, y_pred, self.save_results + self.matrix_path, self.classes)
            if self.roc_path:
                save_roc(y_true, y_proba, self.save_results + self.roc_path, len(self.classes))

        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

### Define the client_common function to set up the Flower client

In [6]:
def client_common(cid, model_save, path_yaml, path_roc, results_save, path_matrix,
                  batch_size, trainloaders, valloaders, DEVICE, CLASSES,
                  he=False, secret_path="", server_path=""):
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    context_client = None
    net = Net(num_classes=len(CLASSES)).to(DEVICE)

    if he:
        print("Run with homomorphic encryption")
        if os.path.exists(secret_path):
            with open(secret_path, 'rb') as f:
                query = pickle.load(f)
            context_client = ts.context_from(query["contexte"])
        else:
            context_client = security.context()
            with open(secret_path, 'wb') as f:
                encode = pickle.dumps({"contexte": context_client.serialize(save_secret_key=True)})
                f.write(encode)
        secret_key = context_client.secret_key()
    else:
        print("Run WITHOUT homomorphic encryption")

    if os.path.exists(model_save):
        print(" To get the checkpoint")
        checkpoint = torch.load(model_save, map_location=DEVICE)['model_state_dict']
        if he:
            print("to decrypt model")
            server_query, server_context = security.read_query(server_path)
            server_context = ts.context_from(server_context)
            for name in checkpoint:
                print(name)
                checkpoint[name] = torch.tensor(
                    security.deserialized_layer(name, server_query[name], server_context).decrypt(secret_key)
                )
        net.load_state_dict(checkpoint)

    return FlowerClient(cid, net, trainloader, valloader, device=DEVICE, batch_size=batch_size,
                        matrix_path=path_matrix, roc_path=path_roc, save_results=results_save, yaml_path=path_yaml,
                        he=he, context_client=context_client, classes=CLASSES)

### Define utility functions for federated learning

In [7]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]
    return {"accuracy": sum(accuracies) / sum(examples)}

def evaluate2(server_round: int, parameters: NDArrays,
              config: Dict[str, Scalar]) -> Optional[Tuple[float, Dict[str, Scalar]]]:
    set_parameters(central, parameters)
    loss, accuracy, y_pred, y_true, y_proba = engine.test(central, testloader, loss_fn=torch.nn.CrossEntropyLoss(),
                                                          device=DEVICE)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

def get_on_fit_config_fn(epoch=2, lr=0.001, batch_size=32) -> Callable[[int], Dict[str, str]]:
    def fit_config(server_round: int) -> Dict[str, str]:
        config = {
            "learning_rate": str(lr),
            "batch_size": str(batch_size),
            "server_round": server_round,
            "local_epochs": epoch
        }
        return config
    return fit_config

def aggreg_fit_checkpoint(server_round, aggregated_parameters, central_model, path_checkpoint,
                          context_client=None, server_path=""):
    if aggregated_parameters is not None:
        print(f"Saving round {server_round} aggregated_parameters...")
        aggregated_ndarrays: List[np.ndarray] = parameters_to_ndarrays_custom(aggregated_parameters, context_client)
        if context_client:   
            server_response = {"contexte": server_context.serialize()}
            for i, key in enumerate(central_model.state_dict().keys()):
                try:
                    server_response[key] = aggregated_ndarrays[i].serialize()
                except:
                    server_response[key] = aggregated_ndarrays[i]
            security.write_query(server_path, server_response)
        else:
            params_dict = zip(central_model.state_dict().keys(), aggregated_ndarrays)
            state_dict = OrderedDict({k: torch.tensor(v) for k, v in params_dict})
            central_model.load_state_dict(state_dict, strict=True)
            if path_checkpoint:
                torch.save({
                    'model_state_dict': central_model.state_dict(),
                }, path_checkpoint)

### Define the FedCustom strategy class

In [8]:
# A Strategy from scratch with the same sampling of the clients as it is in FedAvg
# and then change the configuration dictionary
class FedCustom(fl.server.strategy.Strategy):
    def __init__(
            self,
            fraction_fit: float = 1.0,
            fraction_evaluate: float = 1.0,
            min_fit_clients: int = 2,
            min_evaluate_clients: int = 2,
            min_available_clients: int = 2,
            evaluate_fn: Optional[
                    Callable[[int, NDArrays, Dict[str, Scalar]], Optional[Tuple[float, Dict[str, Scalar]]]]
                ] = None,
            on_fit_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
            on_evaluate_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
            accept_failures: bool = True,
            initial_parameters: Optional[Parameters] = None,
            fit_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
            evaluate_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
            context_client=None
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn,
        self.accept_failures = accept_failures
        self.initial_parameters = initial_parameters
        self.fit_metrics_aggregation_fn = fit_metrics_aggregation_fn
        self.evaluate_metrics_aggregation_fn = evaluate_metrics_aggregation_fn
        self.context_client = context_client

    def __repr__(self) -> str:
        # Same function as FedAvg(Strategy)
        return f"FedCustom (accept_failures={self.accept_failures})"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        # Same function as FedAvg(Strategy)
        initial_parameters = self.initial_parameters
        self.initial_parameters = None  # Don't keep initial parameters in memory
        return initial_parameters

    def num_fit_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Return sample size and required number of clients."""
        # Same function as FedAvg(Strategy)
        num_clients = int(num_available_clients * self.fraction_fit)
        return max(num_clients, self.min_fit_clients), self.min_available_clients

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""
        # Sample clients
        sample_size, min_num_clients = self.num_fit_clients(
            client_manager.num_available()
        )

        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )
        # Create custom configs
        n_clients = len(clients)
        half_clients = n_clients // 2
        # Custom fit config function provided
        standard_lr = lr
        higher_lr = 0.003
        config = {"server_round": server_round, "local_epochs": 1}
        if self.on_fit_config_fn is not None:
            # Custom fit config function provided
            config = self.on_fit_config_fn(server_round)

        # fit_ins = FitIns(parameters, config)
        # Return client/config pairs
        fit_configurations = []
        for idx, client in enumerate(clients):
            config["learning_rate"] = standard_lr if idx < half_clients else higher_lr
            """
            Each pair of (ClientProxy, FitRes) constitutes 
            a successful update from one of the previously selected clients.
            """
            fit_configurations.append(
                (
                    client,
                    FitIns(
                        parameters,
                        config
                    )
                )
            )
        # Successful updates from the previously selected and configured clients
        return fit_configurations

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average. (each round)"""
        # Same function as FedAvg(Strategy)
        if not results:
            return None, {}

        # Do not aggregate if there are failures and failures are not accepted
        if not self.accept_failures and failures:
            return None, {}

        # Convert results parameters --> array matrix
        weights_results = [
            (parameters_to_ndarrays_custom(fit_res.parameters, self.context_client), fit_res.num_examples)
            for _, fit_res in results
        ]

        # Aggregate parameters using weighted average between the clients and convert back to parameters object (bytes)
        parameters_aggregated = ndarrays_to_parameters_custom(aggregate_custom(weights_results))

        metrics_aggregated = {}
        # Aggregate custom metrics if aggregation fn was provided
        if self.fit_metrics_aggregation_fn:
            fit_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.fit_metrics_aggregation_fn(fit_metrics)

        elif server_round == 1:  # Only log this warning once
            logger.log(WARNING, "No fit_metrics_aggregation_fn provided")

        # Same function as SaveModelStrategy(fl.server.strategy.FedAvg)
        """Aggregate model weights using weighted average and store checkpoint"""
        aggreg_fit_checkpoint(server_round, parameters_aggregated, central, model_save,
                              self.context_client, path_crypted)
        return parameters_aggregated, metrics_aggregated

    def num_evaluation_clients(self, num_available_clients: int) -> Tuple[int, int]:
        """Use a fraction of available clients for evaluation."""
        # Same function as FedAvg(Strategy)
        num_clients = int(num_available_clients * self.fraction_evaluate)
        return max(num_clients, self.min_evaluate_clients), self.min_available_clients

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""
        # Same function as FedAvg(Strategy)
        # Do not configure federated evaluation if fraction eval is 0.
        if self.fraction_evaluate == 0.0:
            return []

        # Parameters and config
        config = {}  # {"server_round": server_round, "local_epochs": 1}

        evaluate_ins = EvaluateIns(parameters, config)

        # Sample clients
        sample_size, min_num_clients = self.num_evaluation_clients(
            client_manager.num_available()
        )

        clients = client_manager.sample(
            num_clients=sample_size, min_num_clients=min_num_clients
        )

        # Return client/config pairs
        # Each pair of (ClientProxy, FitRes) constitutes a successful update from one of the previously selected clients
        return [(client, evaluate_ins) for client in clients]

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""
        # Same function as FedAvg(Strategy)
        if not results:
            return None, {}

        # Do not aggregate if there are failures and failures are not accepted
        if not self.accept_failures and failures:
            return None, {}

        # Aggregate loss
        loss_aggregated = weighted_loss_avg(
            [
                (evaluate_res.num_examples, evaluate_res.loss)
                for _, evaluate_res in results
            ]
        )

        metrics_aggregated = {}
        # Aggregate custom metrics if aggregation fn was provided
        if self.evaluate_metrics_aggregation_fn:
            eval_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.evaluate_metrics_aggregation_fn(eval_metrics)

        # Only log this warning once
        elif server_round == 1:
            logger.log(WARNING, "No evaluate_metrics_aggregation_fn provided")

        return loss_aggregated, metrics_aggregated

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate global model parameters using an evaluation function."""
        # Same function as FedAvg(Strategy)
        if self.evaluate_fn is None:
            # Let's assume we won't perform the global model evaluation on the server side.
            return None

        # if we have a global model evaluation on the server side :
        parameters_ndarrays = parameters_to_ndarrays_custom(parameters, self.context_client)
        eval_res = self.evaluate_fn(server_round, parameters_ndarrays, {})

        # if you haven't results
        if eval_res is None:
            return None

        loss, metrics = eval_res
        return loss, metrics

### Set up the federated learning strategy

In [9]:
# Set up your variables directly
he = True
data_path = 'data/'
dataset = 'PCOS'
yaml_path = './results/FL/results.yml'
seed = 0
num_workers = 0
max_epochs = 10
batch_size = 32
splitter = 10
device = 'gpu'
number_clients = 10
save_results = 'results/FL/'
matrix_path = 'confusion_matrix.png'
roc_path = 'roc.png'
model_save = 'PCOS_FHE.pt'
min_fit_clients = 10
min_avail_clients = 10
min_eval_clients = 10
rounds = 20
frac_fit = 1.0
frac_eval = 0.5
lr = 1e-3
path_public_key = 'server_key.pkl'

In [10]:
print("get public key : ", path_public_key)
_, server_context = security.read_query(path_public_key)
server_context = ts.context_from(server_context)
DEVICE = torch.device(choice_device(device))
CLASSES = classes_string(dataset)
central = Net(num_classes=len(CLASSES)).to(DEVICE)

get public key :  server_key.pkl


In [11]:
strategy = FedCustom(
    fraction_fit=frac_fit,
    fraction_evaluate=frac_eval,
    min_fit_clients=min_fit_clients,
    min_evaluate_clients=min_eval_clients if min_eval_clients else number_clients // 2,
    min_available_clients=min_avail_clients,
    evaluate_metrics_aggregation_fn=weighted_average,
    initial_parameters=ndarrays_to_parameters_custom(get_parameters2(central)),
    evaluate_fn=None if he else evaluate2,
    on_fit_config_fn=get_on_fit_config_fn(epoch=max_epochs, batch_size=batch_size),
    context_client=server_context
)

In [ ]:
trainloaders, valloaders, testloader = data_setup.load_datasets(num_clients=number_clients,
                                                                batch_size=batch_size,
                                                                resize=224,
                                                                seed=seed,
                                                                num_workers=num_workers,
                                                                splitter=splitter,
                                                                dataset=dataset,  # Use the specified dataset
                                                                data_path=data_path,
                                                                data_path_val=None)  # Use the same path for validation data

def client_fn(cid: str) -> FlowerClient:
    return client_common(cid,
                         model_save, path_yaml, path_roc, results_save, path_matrix,
                         batch_size, trainloaders, valloaders, DEVICE, CLASSES, he, secret_path, server_path)

PCOS
The training set is created for the classes : ['0', '1']


### Define the client_fn function and set up the simulation

In [ ]:
import warnings
warnings.simplefilter("ignore")

print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)
print("torchvision", torchvision.__version__)
print(f"Training on {DEVICE}")

client_resources = None

if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

model_save = model_save
path_yaml = yaml_path
path_roc = roc_path
results_save = save_results
path_matrix = matrix_path
batch_size = batch_size
he = he
secret_path = 'secret.pkl'
server_path = 'secret.pkl'
path_crypted = 'server.pkl'

print("Start simulation")
start_simulation = time.time()
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=number_clients,
    config=fl.server.ServerConfig(num_rounds=rounds),
    strategy=strategy,
    client_resources=client_resources
)
print(f"Simulation Time = {time.time() - start_simulation} seconds")

INFO flwr 2024-07-24 04:16:06,552 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=20, round_timeout=None)


flwr 1.5.0
numpy 1.26.4
torch 2.1.2
torchvision 0.16.2
Training on cuda:0
Start simulation


2024-07-24 04:16:11,202	INFO worker.py:1724 -- Started a local Ray instance.
INFO flwr 2024-07-24 04:16:12,604 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 8917659648.0, 'node:172.19.2.2': 1.0, 'node:__internal_head__': 1.0, 'memory': 17835319296.0, 'GPU': 2.0, 'accelerator_type:T4': 1.0, 'CPU': 4.0}
INFO flwr 2024-07-24 04:16:12,607 | server.py:89 | Initializing global parameters
INFO flwr 2024-07-24 04:16:12,608 | server.py:272 | Using initial parameters provided by strategy
INFO flwr 2024-07-24 04:16:12,613 | server.py:91 | Evaluating initial parameters
INFO flwr 2024-07-24 04:16:12,614 | server.py:104 | FL starting
DEBUG flwr 2024-07-24 04:16:12,615 | server.py:222 | fit_round 1: strategy sampled 10 clients (out of 10)


(launch_and_fit pid=407) Run with homomorphic encryption
(launch_and_fit pid=407) [Client 5, round 1] fit, config: {'learning_rate': 0.003, 'batch_size': '32', 'server_round': 1, 'local_epochs': 10}
(launch_and_fit pid=407) Updated model


  0%|          | 0/10 [00:00<?, ?it/s]


...

Saving round 20 aggregated_parameters...


DEBUG flwr 2024-07-24 05:41:59,583 | server.py:173 | evaluate_round 20: strategy sampled 10 clients (out of 10)


(launch_and_evaluate pid=16412) Run with homomorphic encryption
(launch_and_fit pid=16346) save graph in  results/FL/
(launch_and_fit pid=16346) 	Train Epoch: 10 	Train_loss: 0.0052 | Train_acc: 99.6528 % | Validation_loss: 0.6825 | Validation_acc: 85.7143 %
(launch_and_fit pid=16346) classifier.4.bias 9.5367431640625e-07 [repeated 11x across cluster]
(launch_and_evaluate pid=16412) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=16413) Updated model
(launch_and_evaluate pid=16490) Run with homomorphic encryption [repeated 2x across cluster]
(launch_and_evaluate pid=16413) [Client 7] evaluate, config: {}
(launch_and_evaluate pid=16412) Updated model
(launch_and_evaluate pid=16490) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=16490) Updated model
(launch_and_evaluate pid=16568) Run with homomorphic encryption [repeated 2x across cluster]
(launch_and_evaluate pid=16492) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=16492) Updated model
(launch_and_evaluate

DEBUG flwr 2024-07-24 05:43:20,898 | server.py:187 | evaluate_round 20 received 10 results and 0 failures
INFO flwr 2024-07-24 05:43:20,900 | server.py:153 | FL finished in 5228.284503534001
INFO flwr 2024-07-24 05:43:20,944 | app.py:225 | app_fit: losses_distributed [(1, 0.75010946393013), (2, 0.6826132714748383), (3, 0.6408450305461884), (4, 0.6391132771968842), (5, 0.6532269775867462), (6, 0.5813766837120056), (7, 0.6655099034309387), (8, 0.7934143871068955), (9, 0.8839525878429413), (10, 0.9120015412569046), (11, 0.9659294337034225), (12, 0.9250698208808898), (13, 0.9495759069919586), (14, 1.0799603044986725), (15, 1.0515673816204072), (16, 1.009228527545929), (17, 1.090744137763977), (18, 1.1891956746578216), (19, 1.0945355534553527), (20, 1.306697964668274)]
INFO flwr 2024-07-24 05:43:20,945 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2024-07-24 05:43:20,946 | app.py:227 | app_fit: metrics_distributed {'accuracy': [(1, 40.59113300492611), (2, 57.266009852216754),

Simulation Time = 5234.403892755508 seconds
